# The Cancer Gene Atlas Cancer (TCGA) - Adrenocortical Carcinoma (ACC)

Author: Moshe Silverstein <br/>
Date: 7-17 <br/>
Data Source: https://cancergenome.nih.gov/

In [1]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import matplotlib.pyplot as plt
%matplotlib inline
from collections import Counter

In [2]:
import seaborn as sns
sns.set(color_codes=True)
np.random.seed(sum(map(ord, "distributions")))

In [3]:
from clustergrammer_widget import *
net = Network(clustergrammer_widget)

In [4]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/TCGA/untility_functions.py'>

# Versions Of Modules In Use

In [5]:
%load_ext version_information
%version_information numpy, pandas, clustergrammer_widget 

Software versions
Python 3.5.2 64bit [GCC 4.2.1 Compatible Apple LLVM 4.2 (clang-425.0.28)]
IPython 5.3.0
OS Darwin 17.5.0 x86_64 i386 64bit
numpy 1.13.1
pandas 0.21.0
clustergrammer_widget 1.9.0
Fri Apr 20 12:53:11 2018 EDT

# Functions

In [6]:
def getGenes(inputDF):
    
    inputDF.reset_index(inplace=True)
    
    inputDF.rename(columns={'index':'Gene Symbol'}, inplace=True)

    inputDF.set_index('Gene Symbol', inplace=True)

    lst = []

    for index in  inputDF.index:
        lst.append(index.split('|')[5])

    inputDF.index = lst

In [7]:
def getSampleID(inputDF, metaDF):
    
    lst = []
    
    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()

        ID = ('-').join(metaDF.loc[col, 'a_AliquotBarcode'].split('-')[0:4])

        lst.append(ID)

        
    inputDF.columns = lst

In [8]:
def getSampleData(inputDF, metaDF):
    
    lst = []

    for i,col in enumerate(inputDF.columns):

        progressPercent = ((i+1)/len(inputDF.columns))*100

        sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(inputDF.columns)))
        sys.stdout.flush()
        
#         ID = ('-').join(meta.loc[col, 'a_AliquotBarcode'].split('-')[0:3])
        ID = ('-').join(col.split('-')[0:3])
        
        donor = 'Donor:'+ID
        
        if str(col.split('-')[3]) == '01A':
            sampleType = 'Primary Solid Tumor'
        elif str(col.split('-')[3]) == '11A':
            sampleType= 'Solid Tissue Normal'
        
        if ID in metaDF.index:
            info = (donor, 
                'Age:'+str(metaDF.loc[ID, 'age_at_initial_pathologic_diagnosis']),
                'Gender:'+str(metaDF.loc[ID, 'gender']),    
                'SampleType:'+sampleType,
                'Stage:'+str(metaDF.loc[ID, 'pathologic_stage']),
                'Histology:'+str(metaDF.loc[ID, 'histological_type']),
                'Tissue:'+str(metaDF.loc[ID, 'tumor_tissue_site']), 
                'Tumor:'+str(metaDF.loc[ID, 'tumor_type']),    
                'Grade'+str(metaDF.loc[ID, 'neoplasm_histologic_grade'])
#                 'SampleType:'+str(ID.split('-')[3])
               )
#             for col in metaDF.columns:
#                 if len(metaDF[col].dropna()) >= .5*len(metaDF[col]) and len(metaDF[col].unique()) > 2 and len(metaDF[col].unique()) < len(metaDF[col]) and 'age' not in col:
#                     info = info + (str(col)+':'+str(metaDF.loc[ID, col]),)
        lst.append(info)

        
    inputDF.columns = lst

# Path to Output Files

In [9]:
path = '/Users/moshesilverstein/Documents/Harmonizome/TCGA/Output/'

# Load Meta Data

In [10]:
meta = pd.read_csv('Input/TCGA_Metadata.csv', index_col=9)

In [11]:
meta.head()

,a_AliquotBarcode,b_disease,a_objectSizeBytes,b_uploaded,b_state,b_library_type,b_center,b_assembly,a_GCSobject
a_CGHubAnalysisID,,,,,,,,,
3a8e6a74-137f-468f-8987-fa0acdde2836,TCGA-CS-6188-01A-11R-1896-07,LGG,6354301047,8/14/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
c63d44f1-c9ad-4391-b1aa-0feb3713441b,TCGA-EO-A22Y-01A-11R-A180-07,UCEC,5497637066,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
21912089-1e42-4bcc-9ad9-fe9a9b88fb09,TCGA-94-A5I4-01A-11R-A26W-07,LUSC,4600317850,8/29/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
6b86e544-0372-434a-9f28-6fa4075dd228,TCGA-N5-A4RV-01A-21R-A28V-07,UCS,4964387768,8/30/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...
30518eb4-2783-4988-bcd4-36a1a8bb4dfa,TCGA-BK-A56F-01A-32R-A27V-07,UCEC,5668623460,8/31/2013,Live,RNA-Seq,UNC-LCCC,unaligned,gs://5aa919de-0aa0-43ec-9ec3-288481102b6d/tcga...


In [12]:
meta.shape

(11373, 9)

# Load Sample MetaData

In [13]:
sample_meta = pd.read_csv('Input/metadata_Adrenocortical carcinoma [ACC]', sep='\t', index_col=1)

In [14]:
sample_meta.head()

,Unnamed: 0,Study,Project,ParticipantUUID,TSSCode,age_at_initial_pathologic_diagnosis,anatomic_neoplasm_subdivision,batch_number,bcr,clinical_M,...,BMI,age_began_smoking_in_years,h_pylori_infection,other_dx,other_malignancy_anatomic_site,other_malignancy_histological_type,other_malignancy_malignancy_type,stopped_smoking_year,venous_invasion,year_of_tobacco_smoking_onset
ParticipantBarcode,,,,,,,,,,,,,,,,,,,,,
TCGA-OR-A5K3,10480,ACC,TCGA,C72B973B-4719-46AB-AC99-05D14088D3BC,OR,53.0,NaN,304,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-OR-A5KV,10481,ACC,TCGA,202F7A77-9E5A-4F12-8FFE-78AE5527A746,OR,17.0,NaN,304,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-OR-A5L5,10482,ACC,TCGA,2F04E760-8A64-49F2-A1D6-3D390604745C,OR,77.0,NaN,304,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,"Yes, History of Synchronous/Bilateral Malignancy",Ovary,"Other, specify",Synchronous Malignancy,NaN,NaN,NaN
TCGA-OR-A5KX,10483,ACC,TCGA,33C02ECE-911B-4F45-B410-9DF07E9B189E,OR,25.0,NaN,304,Nationwide Children's Hospital,M0,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN
TCGA-OR-A5JB,10484,ACC,TCGA,D63F2F1E-1970-445F-907B-23CF4D9EFD83,OR,52.0,NaN,304,Nationwide Children's Hospital,M1,...,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
sample_meta.shape

(92, 70)

# Load Data

In [16]:
matrix = pd.read_csv('Input/TCGA_ACC_tpm.tsv', sep='\t', index_col=0)

In [17]:
matrix.head()

,ca6d9c83-f6bc-474a-a87a-04a8d64f6752,e6f11961-dfda-4c6e-b051-54b27ad9d0de,25de321c-8ad7-4882-8c04-056649e9b4e5,80653994-a361-4e45-b044-502ce874a9d8,8c0b0a9b-6066-4bc3-8f09-1e0454e8b8a3,ec870197-fd7c-405b-bcdf-03c309ed4274,f33c9ee1-d32c-4606-b264-99bcd01e2059,0009f27d-a053-49b0-a030-ba9c1469224f,9b75d36c-9d68-45c1-8a47-bde800bf287e,8e4b078b-e6f7-4424-95fd-51a550203835,...,a8b9b341-f75d-4711-b7d9-667fa7d7efeb,e996049f-b7a5-4e57-b67f-4aa802c671ea,31767219-2d21-4258-86d9-ad0ef4a507f9,cc0f11a0-c323-422a-88f9-7238e050ad73,1b43c9ef-bae9-4160-9185-e1d81bd3c4fb,e03dc734-759e-4aec-a071-279ab599fb7b,c096d69c-d628-46bd-b553-9e90f73c8fb2,d2c3e0e7-bc22-435e-974c-a175e10d7542,f3b57849-0302-47b0-9e3b-7cbe9de07056,8bf5ee5d-fc34-4305-9e23-f9d999e8a64c
ENST00000000233.9|ENSG00000004059.10|OTTHUMG00000023246.6|OTTHUMT00000059567.2|ARF5-001|ARF5|1103|protein_coding|,139.56300,152.81100,78.854900,407.21200,173.95600,160.92800,81.53210,137.521000,272.265000,91.5068,...,221.2150,164.633000,109.46500,168.300000,1.666770e+02,187.81800,117.1930,85.76750,105.5070,188.308000
ENST00000000412.7|ENSG00000003056.7|OTTHUMG00000168276.2|OTTHUMT00000399130.1|M6PR-001|M6PR|2756|protein_coding|,39.48070,8.00477,20.878100,1.48273,30.70750,36.89430,20.10400,13.871800,27.336000,36.1530,...,58.2828,43.083100,33.01430,25.820600,5.660110e+00,37.79450,23.4717,20.58830,86.6886,32.430200
ENST00000000442.10|ENSG00000173153.13|OTTHUMG00000150641.6|OTTHUMT00000319303.1|ESRRA-002|ESRRA|2215|protein_coding|,15.78650,25.01430,24.295800,41.15060,11.51740,40.52380,20.28820,18.515200,28.573800,39.1969,...,23.5859,12.906400,26.79040,33.599500,1.372730e+01,28.93700,14.1818,7.59664,21.5032,11.909600
ENST00000001008.5|ENSG00000004478.7|OTTHUMG00000090429.3|OTTHUMT00000206861.2|FKBP4-001|FKBP4|3732|protein_coding|,113.86700,13.45580,30.066700,41.50720,57.47680,41.25560,38.37850,13.410100,19.983700,13.5083,...,30.3721,39.656500,29.74550,13.497900,4.999490e+01,13.19950,59.1308,45.94380,55.6202,40.776200
ENST00000001146.6|ENSG00000003137.8|OTTHUMG00000129756.5|OTTHUMT00000251969.1|CYP26B1-001|CYP26B1|4732|protein_coding|,5.13914,0.23997,0.049712,0.00000,0.26945,2.17208,0.14382,0.119374,0.382538,12.1989,...,0.0000,0.074344,1.71667,0.499127,4.939720e-08,4.48836,11.9792,0.00000,1.4864,0.138578


In [18]:
matrix.shape

(199169, 79)

# Get Only Protein Coding Genes

In [19]:
lst = []

for i,index in enumerate(matrix.index):
    
    progressPercent = ((i+1)/len(matrix.index)*100)

    sys.stdout.write("Progress: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(matrix.index)))
    sys.stdout.flush()
                                 
    lst.append(index.split('|')[7])    
                       
matrix['Gene Type'] = lst

matrix = matrix[matrix['Gene Type'] == 'protein_coding']

matrix.drop('Gene Type', axis=1, inplace=True)

In [20]:
matrix.head()

In [21]:
matrix.shape 

# Map Sample Meta Data to Sample ID

In [22]:
getSampleID(matrix, meta)

In [23]:
matrix.head()

# Map Gene Symbol to Gene ID

In [24]:
getGenes(matrix)

In [25]:
matrix.head()

# Histogram of First Sample

In [26]:
# matrix.iloc[:, 0].hist(bins=100, log=True);
sns.distplot(matrix.iloc[:, 0]);

# Histogram of First Gene

In [27]:
# matrix.iloc[0, :].hist(bins=100);
sns.distplot(matrix.iloc[0, :]);

# Save Unfiltered Matrix to File

In [28]:
filename = path+'tcga_acc_matrix_unfilltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
matrix.to_csv(filename, sep='\t', compression='gzip')

# Filtered

In [29]:
normalized_matrix = matrix.copy()

In [30]:
normalized_matrix = normalized_matrix.groupby(level=0).max()

# Remove any data with more than 95% missing and imput remaining mssing values to matrix mean

In [31]:
normalized_matrix = uf.removeAndImpute(normalized_matrix)

In [32]:
normalized_matrix.head()

,TCGA-OR-A5JG-01A,TCGA-OR-A5LG-01A,TCGA-OR-A5JD-01A,TCGA-OR-A5LH-01A,TCGA-OR-A5KY-01A,TCGA-PK-A5HB-01A,TCGA-PK-A5HA-01A,TCGA-OR-A5KX-01A,TCGA-PK-A5H8-01A,TCGA-PK-A5H9-01A,...,TCGA-OR-A5JL-01A,TCGA-OR-A5JW-01A,TCGA-OR-A5JM-01A,TCGA-OR-A5JO-01A,TCGA-OR-A5JX-01A,TCGA-OR-A5JY-01A,TCGA-OR-A5K9-01A,TCGA-OR-A5K6-01A,TCGA-OR-A5J2-01A,TCGA-OR-A5J3-01A
A1BG,0.156866,0.041220,0.041220,0.041220,0.041220,0.041220,0.028512,0.023254,0.022345,0.041220,...,0.041220,0.041220,0.041220,0.041220,0.026223,0.041220,0.041220,0.041220,0.041220,0.047193
A1CF,0.014681,0.013787,0.007410,0.073812,0.025045,0.081300,0.006817,0.011464,0.002801,0.430081,...,0.013082,0.091919,0.011600,0.270317,0.062173,0.015945,0.002572,0.009528,0.025045,0.008615
A2M,63.791800,60.448400,44.173500,106.658000,82.479000,146.937000,104.417000,42.631700,59.316500,4005.930000,...,75.113700,76.831600,31.308200,1287.360000,72.070700,73.939900,38.041300,39.414800,105.559000,62.771700
A2ML1,0.098411,0.048085,0.218348,0.749764,0.190570,0.035602,0.021462,0.116991,0.291238,0.263584,...,0.667841,0.130362,0.015975,0.080181,0.782357,0.085386,0.084290,0.083801,0.035500,0.107645
A3GALT2,0.071749,0.071749,0.071749,0.071749,0.071749,0.045591,0.071749,0.071749,0.049750,0.022483,...,0.071749,0.151256,0.106385,0.071749,0.019416,1.067380,0.071749,0.021218,0.071749,0.068970


In [33]:
normalized_matrix.shape

(18942, 79)

# logCPM Transform

Raw counts were normalized to log10-Counts Per Million (logCPM) by dividing each column by the total sum of its counts, multiplying it by 106, followed by the application of a log10-transform.

In [34]:
for col in normalized_matrix.columns:

    total = normalized_matrix[col].sum()
    
    normalized_matrix[col] = normalized_matrix[col].apply(lambda x: np.log10(((x)/total)*10**6))

In [35]:
normalized_matrix.head()

,TCGA-OR-A5JG-01A,TCGA-OR-A5LG-01A,TCGA-OR-A5JD-01A,TCGA-OR-A5LH-01A,TCGA-OR-A5KY-01A,TCGA-PK-A5HB-01A,TCGA-PK-A5HA-01A,TCGA-OR-A5KX-01A,TCGA-PK-A5H8-01A,TCGA-PK-A5H9-01A,...,TCGA-OR-A5JL-01A,TCGA-OR-A5JW-01A,TCGA-OR-A5JM-01A,TCGA-OR-A5JO-01A,TCGA-OR-A5JX-01A,TCGA-OR-A5JY-01A,TCGA-OR-A5K9-01A,TCGA-OR-A5K6-01A,TCGA-OR-A5J2-01A,TCGA-OR-A5J3-01A
A1BG,-0.638711,-1.202029,-1.242035,-1.145422,-1.206982,-1.209328,-1.441934,-1.440753,-1.449663,-1.211022,...,-1.203244,-1.219899,-1.194185,-1.205777,-1.414634,-1.212832,-1.199554,-1.226342,-1.223599,-1.171409
A1CF,-1.667490,-1.677671,-1.987335,-0.892406,-1.423372,-0.914349,-2.063395,-1.747920,-2.351517,-0.192584,...,-1.701685,-0.871605,-1.744843,-0.389015,-1.039719,-1.625319,-2.404476,-1.862456,-1.439989,-1.910019
A2M,1.970525,1.964244,1.788015,2.267459,2.094249,2.342691,2.121813,1.822476,1.974326,3.776569,...,2.057363,2.050529,1.686361,3.288811,2.024438,2.040935,1.765589,1.754205,2.184785,1.952479
A2ML1,-0.841196,-1.135131,-0.517998,0.114390,-0.542039,-1.272972,-1.565282,-0.739104,-0.334601,-0.405215,...,0.006317,-0.719860,-1.605848,-0.916816,0.060084,-0.896557,-0.888890,-0.918203,-1.288481,-0.813291
A3GALT2,-0.978422,-0.961323,-1.001329,-0.904716,-0.966276,-1.165565,-1.041140,-0.951438,-1.102056,-1.474284,...,-0.962538,-0.655299,-0.782417,-0.965071,-1.545168,0.200375,-0.958848,-1.514756,-0.982893,-1.006628


# Normalize Matrix (Quantile Normalize the matrix for the columns)

In [36]:
normalized_matrix = uf.quantileNormalize(normalized_matrix)

In [37]:
normalized_matrix.head()

,TCGA-OR-A5JG-01A,TCGA-OR-A5LG-01A,TCGA-OR-A5JD-01A,TCGA-OR-A5LH-01A,TCGA-OR-A5KY-01A,TCGA-PK-A5HB-01A,TCGA-PK-A5HA-01A,TCGA-OR-A5KX-01A,TCGA-PK-A5H8-01A,TCGA-PK-A5H9-01A,...,TCGA-OR-A5JL-01A,TCGA-OR-A5JW-01A,TCGA-OR-A5JM-01A,TCGA-OR-A5JO-01A,TCGA-OR-A5JX-01A,TCGA-OR-A5JY-01A,TCGA-OR-A5K9-01A,TCGA-OR-A5K6-01A,TCGA-OR-A5J2-01A,TCGA-OR-A5J3-01A
A1BG,-0.571788,-1.145362,-1.079838,-1.460520,-1.217320,-1.262630,-1.277548,-1.439316,-1.509637,-1.271298,...,-1.250523,-1.247208,-1.237270,-1.263803,-1.375212,-1.293831,-1.193047,-1.194534,-1.220309,-1.188571
A1CF,-1.625907,-1.606723,-1.791546,-1.202049,-1.421487,-0.973647,-1.826819,-1.728389,-2.359489,-0.342735,...,-1.754250,-0.914330,-1.814707,-0.523019,-0.998524,-1.707977,-2.271921,-1.827601,-1.426402,-1.938644
A2M,1.924820,1.907796,2.061700,2.154738,2.024303,2.173065,2.366827,1.761977,1.928294,3.617204,...,1.920462,1.992277,1.470176,3.073233,2.089256,1.924820,1.639667,1.715326,2.175782,1.918706
A2ML1,-0.789614,-1.068842,-0.334305,-0.130596,-0.566594,-1.320414,-1.405082,-0.741531,-0.415895,-0.535167,...,-0.085592,-0.762326,-1.658301,-0.987228,0.142831,-0.994366,-0.880288,-0.884127,-1.287465,-0.836279
A3GALT2,-0.928499,-0.891094,-0.831436,-1.214898,-0.981177,-1.220821,-0.866586,-0.950115,-1.172615,-1.518025,...,-1.014756,-0.696508,-0.817078,-1.040125,-1.510091,0.058165,-0.949605,-1.477832,-0.990521,-1.024335


# Normalize Matrix (z-score the rows)

In [38]:
uf.zscore(normalized_matrix, 'row')

In [39]:
normalized_matrix.head()

,TCGA-OR-A5JG-01A,TCGA-OR-A5LG-01A,TCGA-OR-A5JD-01A,TCGA-OR-A5LH-01A,TCGA-OR-A5KY-01A,TCGA-PK-A5HB-01A,TCGA-PK-A5HA-01A,TCGA-OR-A5KX-01A,TCGA-PK-A5H8-01A,TCGA-PK-A5H9-01A,...,TCGA-OR-A5JL-01A,TCGA-OR-A5JW-01A,TCGA-OR-A5JM-01A,TCGA-OR-A5JO-01A,TCGA-OR-A5JX-01A,TCGA-OR-A5JY-01A,TCGA-OR-A5K9-01A,TCGA-OR-A5K6-01A,TCGA-OR-A5J2-01A,TCGA-OR-A5J3-01A
A1BG,9.341924,1.238522,2.164250,-3.213986,0.221907,-0.418219,-0.628976,-2.914423,-3.907905,-0.540678,...,-0.247178,-0.200343,-0.059936,-0.434792,-2.008766,-0.859028,0.564845,0.543828,0.179678,0.628074
A1CF,0.491016,0.549629,-0.015058,1.786023,1.115577,2.483856,-0.122827,0.177906,-1.750288,4.411473,...,0.098892,2.665089,-0.085823,3.860656,2.407851,0.240268,-1.482741,-0.125218,1.100562,-0.464484
A2M,-0.218752,-0.280010,0.273779,0.608557,0.139216,0.674500,1.371711,-0.804705,-0.206251,5.870909,...,-0.234433,0.023976,-1.854687,3.913551,0.372934,-0.218752,-1.244812,-0.972568,0.684277,-0.240751
A2ML1,-0.052013,-0.584252,0.815856,1.204147,0.373086,-1.063775,-1.225162,0.039638,0.660336,0.432991,...,1.289929,0.000000,-1.707826,-0.428687,1.725328,-0.442293,-0.224848,-0.232165,-1.000971,-0.140962
A3GALT2,0.513198,0.674500,0.931767,-0.721849,0.286033,-0.747391,0.780188,0.419981,-0.539512,-2.029036,...,0.141226,1.513618,0.993680,0.031828,-1.994822,4.768020,0.422180,-1.855709,0.245737,0.099919


# Merge Like Column (by taking the mean)

In [40]:
normalized_matrix = uf.merge(normalized_matrix, 'column', 'mean')

In [41]:
normalized_matrix.shape

(18942, 79)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [42]:
uf.mapgenesymbols(normalized_matrix)

In [43]:
normalized_matrix.shape

(18513, 79)

# Merge Duplicate Genes By Rows

In [44]:
normalized_matrix = uf.merge(normalized_matrix, 'row', 'mean')

In [45]:
normalized_matrix.shape

(18423, 79)

# Histogram of First Sample

In [46]:
# normalized_matrix.iloc[:, 0].hist(bins=100, log=True);
sns.distplot(normalized_matrix.iloc[:, 10]);

# Histogram of First Gene

In [47]:
# normalized_matrix.iloc[1, :].hist(bins=50);
sns.distplot(normalized_matrix.iloc[10, :]);

# Save Filtered Matrix

In [48]:
filename = path+'tcga_acc_matrix_filltered_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
normalized_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [49]:
gene_list = uf.createGeneList(normalized_matrix)

In [50]:
gene_list.head()

,GeneSym,GeneID
0,A1BG,1
1,A1CF,29974
2,A2M,2
3,A2ML1,144568
4,A3GALT2,127550


In [51]:
gene_list.shape

(18423, 2)

# Save Gene List

In [52]:
filename = path+'tcga_acc_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute List

In [53]:
metaData = pd.DataFrame(index=normalized_matrix.columns)

for index in metaData.index:
    metaData.loc[index, 'Age'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'age_at_initial_pathologic_diagnosis']
    metaData.loc[index, 'Gender'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'gender']
    metaData.loc[index, 'Pathologic M'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_M']
    metaData.loc[index, 'Pathologic N'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_N']
    metaData.loc[index, 'Pathologic T'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_T']
    metaData.loc[index, 'Pathologic Stage'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'pathologic_stage']
    metaData.loc[index, 'Histological Type'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'histological_type']
    if str(index.split('-')[3]) == '01A':
        sampleType = 'Primary Solid Tumor'
    elif str(index.split('-')[3]) == '11A':
        sampleType= 'Solid Tissue Normal'
    metaData.loc[index, 'Sample Type'] = sampleType
    metaData.loc[index, 'Tumor Tissue Site'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'tumor_tissue_site']
    metaData.loc[index, 'Neoplasm Histologic Grade'] = sample_meta.loc[('-').join(index.split('-')[0:3]), 'neoplasm_histologic_grade']
    

In [54]:
attribute_list = uf.createAttributeList(normalized_matrix, metaData)

In [55]:
attribute_list.head()

,Age,Gender,Pathologic M,Pathologic N,Pathologic T,Pathologic Stage,Histological Type,Sample Type,Tumor Tissue Site,Neoplasm Histologic Grade
Attributes,,,,,,,,,,
TCGA-OR-A5J1-01A,58,MALE,NaN,N0,T2,Stage II,Adrenocortical carcinoma- Usual Type,Primary Solid Tumor,Adrenal,NaN
TCGA-OR-A5J2-01A,44,FEMALE,NaN,N0,T3,Stage IV,Adrenocortical carcinoma- Usual Type,Primary Solid Tumor,Adrenal,NaN
TCGA-OR-A5J3-01A,23,FEMALE,NaN,N0,T3,Stage III,Adrenocortical carcinoma- Usual Type,Primary Solid Tumor,Adrenal,NaN
TCGA-OR-A5J5-01A,30,MALE,NaN,N0,T4,Stage III,Adrenocortical carcinoma- Usual Type,Primary Solid Tumor,Adrenal,NaN
TCGA-OR-A5J6-01A,29,FEMALE,NaN,N0,T2,Stage II,Adrenocortical carcinoma- Usual Type,Primary Solid Tumor,Adrenal,NaN


In [56]:
attribute_list.shape

(79, 10)

# Save Attribute List

In [57]:
filename = path+'tcga_acc_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', compression='gzip')

# Create matrix of Standardized values (values between -1, and 1)

In [58]:
standard_matrix = uf.createStandardizedMatrix(normalized_matrix)

In [59]:
standard_matrix.head()

,TCGA-OR-A5J1-01A,TCGA-OR-A5J2-01A,TCGA-OR-A5J3-01A,TCGA-OR-A5J5-01A,TCGA-OR-A5J6-01A,TCGA-OR-A5J7-01A,TCGA-OR-A5J8-01A,TCGA-OR-A5J9-01A,TCGA-OR-A5JA-01A,TCGA-OR-A5JB-01A,...,TCGA-OR-A5LS-01A,TCGA-OR-A5LT-01A,TCGA-OU-A5PI-01A,TCGA-P6-A5OF-01A,TCGA-P6-A5OG-01A,TCGA-PA-A5YG-01A,TCGA-PK-A5H8-01A,TCGA-PK-A5H9-01A,TCGA-PK-A5HA-01A,TCGA-PK-A5HB-01A
,,,,,,,,,,,,,,,,,,,,,
A1BG,0.598122,0.142314,0.395397,0.243545,0.522080,0.319472,-0.161531,0.091762,-0.667584,-0.338654,...,-0.996346,0.294127,-0.768764,-0.692871,-0.490443,0.750201,-0.971128,-0.414618,-0.465132,-0.363947
A1CF,-0.482706,0.656535,-0.406835,0.580551,-0.836932,0.327215,0.605862,-0.533270,0.783312,-0.659828,...,0.859371,0.428538,-0.305635,-0.255005,-0.280261,-0.811665,-0.836932,0.986090,-0.103105,0.910018
A2M,0.343441,0.571424,-0.213435,-0.896597,0.140985,0.318141,0.926312,0.672764,-0.162859,-0.871327,...,-0.466463,-0.846018,-0.390596,0.014352,0.850276,0.748856,-0.086932,0.976980,0.824911,0.546114
A2ML1,0.667977,-0.825485,-0.167591,0.161533,-0.699041,-0.142355,-0.420710,0.465252,0.439982,-0.951931,...,0.617306,0.034901,0.744085,0.313334,-0.294191,0.794757,0.414623,0.338657,-0.901350,-0.876076
A3GALT2,-0.912798,0.200669,0.099485,-0.988654,-0.634489,-0.457381,-0.432147,-0.153797,-0.887523,-0.710480,...,-0.027161,0.453811,-0.128504,-0.103105,0.504519,0.707174,-0.204390,-0.761036,0.732641,-0.381514


# Plot of A Single Celltype, Normalized Value vs. Standardized Value

In [60]:
plt.plot(normalized_matrix[normalized_matrix.columns[0]], standard_matrix[standard_matrix.columns[0]], 'bo')
plt.xlabel('Normalized Values')
plt.ylabel('Standardized Values')
plt.title(standard_matrix.columns[0])
plt.grid(True)

# Save Standardized Matrix

In [61]:
filename = path+'tcga_acc_standard_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
standard_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Tertiary Matrix

In [62]:
tertiary_matrix = uf.createTertiaryMatrix(standard_matrix)

In [63]:
tertiary_matrix.head()

,TCGA-OR-A5J1-01A,TCGA-OR-A5J2-01A,TCGA-OR-A5J3-01A,TCGA-OR-A5J5-01A,TCGA-OR-A5J6-01A,TCGA-OR-A5J7-01A,TCGA-OR-A5J8-01A,TCGA-OR-A5J9-01A,TCGA-OR-A5JA-01A,TCGA-OR-A5JB-01A,...,TCGA-OR-A5LS-01A,TCGA-OR-A5LT-01A,TCGA-OU-A5PI-01A,TCGA-P6-A5OF-01A,TCGA-P6-A5OG-01A,TCGA-PA-A5YG-01A,TCGA-PK-A5H8-01A,TCGA-PK-A5H9-01A,TCGA-PK-A5HA-01A,TCGA-PK-A5HB-01A
,,,,,,,,,,,,,,,,,,,,,
A1BG,0,0,0,0,0,0,0,0,0,0,...,-1,0,0,0,0,0,-1,0,0,0
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
A2M,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
A2ML1,0,0,0,0,0,0,0,0,0,-1,...,0,0,0,0,0,0,0,0,0,0
A3GALT2,0,0,0,-1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Save Teriary Matrix

In [64]:
filename = path+'tcga_acc_tertiary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
tertiary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Up Gene Set Library

In [65]:
name = 'tcga_acc_gene_up_set'

In [66]:
uf.createUpGeneSetLib(tertiary_matrix, path, name)

# Create Down Gene Set Library

In [67]:
name = 'tcga_acc_gene_down_set'

In [68]:
uf.createDownGeneSetLib(tertiary_matrix, path, name)

# Create Up Attribute Library

In [69]:
name = 'tcga_acc_attribute_up_set'

In [70]:
uf.createUpAttributeSetLib(tertiary_matrix, path, name)

# Create Down Attribute Library

In [71]:
name = 'tcga_acc_attribute_down_set'

In [72]:
uf.createDownAttributeSetLib(tertiary_matrix, path, name)

# Create Attribute Similarity matrix

In [73]:
attribute_normalized_matrix = normalized_matrix.copy()

In [74]:
getSampleData(attribute_normalized_matrix, sample_meta)

In [75]:
attribute_similarity_matix = uf.createSimilarityMatrix(attribute_normalized_matrix.T, 'cosine')

In [76]:
attribute_similarity_matix.head()

,"(Donor:TCGA-OR-A5J1, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5J2, Age:44.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5J3, Age:23.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5J5, Age:30.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5J6, Age:29.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5J7, Age:30.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5J8, Age:66.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5J9, Age:22.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5JA, Age:53.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5JB, Age:52.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)",...,"(Donor:TCGA-OR-A5LS, Age:34.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OR-A5LT, Age:57.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-OU-A5PI, Age:53.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-P6-A5OF, Age:55.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage III, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-P6-A5OG, Age:45.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage IV, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-PA-A5YG, Age:51.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-PK-A5H8, Age:42.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-PK-A5H9, Age:27.0, Gender:FEMALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-PK-A5HA, Age:63.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage I, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)","(Donor:TCGA-PK-A5HB, Age:63.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:nan, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)"
,,,,,,,,,,,,,,,,,,,,,
"(Donor:TCGA-OR-A5J1, Age:58.0, Gender:MALE, SampleType:Primary Solid Tumor, Stage:Stage II, Histology:Adrenocortical carcinoma- Usual Type, Tissue:Adrenal, Tumor:nan, Gradenan)",1.000000,0.062957,-0.043846,0.080815,0.029129,0.010228,0.034140,0.062756,0.011210,0.053346,...,-0.049717,0.003716,-0.048197,-0.017809,0.082957,-0.045064,0.012252,0.019193,-0.049812,-0.019561
"

# HeatMap (clustergrammer) of Similarity Matrix

In [77]:
net.load_df(attribute_similarity_matix.iloc[:,:].copy())
# net.filter_N_top('row', rank_type='sum', N_top=300)
net.cluster()
net.widget()

Widget Javascript not detected.  It may not be installed or enabled properly.


# Save Attribute Similarity Matrix

In [78]:
filename = path+'tcga_acc_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Similarity Matrix

In [79]:
gene_similarity_matix = uf.createSimilarityMatrix(normalized_matrix, 'cosine')

In [80]:
gene_similarity_matix.head()

,A1BG,A1CF,A2M,A2ML1,A3GALT2,A4GALT,A4GNT,AAAS,AACS,AADAC,...,ZWILCH,ZWINT,ZXDA,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
,,,,,,,,,,,,,,,,,,,,,
A1BG,1.000000,-0.209192,-0.000979,-0.128201,0.177032,-0.156170,0.235723,-0.021756,-0.235740,0.243267,...,-0.170545,-0.082186,-0.241999,-0.198973,-0.271664,-0.294991,-0.238936,0.102319,0.049555,-0.264833
A1CF,-0.209192,1.000000,0.329085,0.021247,-0.132199,0.171632,-0.124970,0.079549,0.076521,0.032595,...,0.148094,0.061908,0.080256,0.170755,0.205849,0.051630,0.174334,0.141525,0.165578,0.107422
A2M,-0.000979,0.329085,1.000000,0.107580,0.041436,0.160231,-0.031248,-0.042285,-0.138630,-0.037913,...,-0.072363,-0.194659,0.105942,0.000461,-0.082203,-0.091180,0.262432,0.354856,-0.057830,0.170772
A2ML1,-0.128201,0.021247,0.107580,1.000000,-0.165897,-0.207178,-0.078741,0.230414,-0.001675,-0.249628,...,-0.221549,-0.286476,0.295641,0.274000,-0.155349,-0.285644,0.037764,-0.054197,-0.005422,0.050536
A3GALT2,0.177032,-0.132199,0.041436,-0.165897,1.000000,-0.217129,0.088365,0.224822,0.027681,0.057266,...,-0.024488,0.038240,-0.084552,-0.034493,-0.072490,-0.156257,-0.036899,-0.173242,-0.040956,-0.092124


# Save Gene Similarity Matrix

In [81]:
filename = path+'tcga_acc_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene-Attribute Edge List

In [82]:
name = 'tcga_acc_gene_attribute_edge_list'

In [83]:
uf.createGeneAttributeEdgeList(standard_matrix, attribute_list, gene_list, path, name)

Progeres: 100%  79 Out of 79   

 The number of statisticaly relevent gene-attribute associations is: 72970
